<a href="https://colab.research.google.com/github/thiagorainmaker77/liar_dataset/blob/master/analise_sentimento_ok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import string
import nltk
from nltk import PorterStemmer
import re
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
from gensim import corpora
import gensim
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\89JBW\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\89JBW\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\89JBW\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
uri_train  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'
uri_valid  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/valid.tsv'
uri_test  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'

    
df_train = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

    
df_valid = pd.read_table(uri_valid,
                             names =['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])


df_test = pd.read_csv(uri_test, sep='\t', 
                            names =['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue']) 


df = pd.concat([df_train, df_valid])

In [3]:
liar_list = [df,df_test]

n = 10

subject = liar_list[0].groupby(['subject']).count().label.sort_values(ascending=False)[:n].index.tolist()
speaker = liar_list[0].groupby(['speaker']).count().label.sort_values(ascending=False)[:n].index.tolist()
job = liar_list[0].groupby(['job']).count().label.sort_values(ascending=False)[:n].index.tolist()
state = liar_list[0].groupby(['state']).count().label.sort_values(ascending=False)[:n].index.tolist()
party = liar_list[0].groupby(['party']).count().label.sort_values(ascending=False)[:n].index.tolist()
venue = liar_list[0].groupby(['venue']).count().label.sort_values(ascending=False)[:n].index.tolist()


In [4]:
class MultiColumnLabelEncoder:
  
  
    countV = CountVectorizer(analyzer='word', binary=True, decode_error='strict', encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
    
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = countV().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = countV().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [5]:
features_of_interest = state+subject+speaker+party+venue+job
to_encode = [('speaker',speaker),
            ('party',party),
            ('subject',subject),
            ('state',state),
            ('venue',venue),
            ('job',job)]
for i in liar_list:
    for var in to_encode:
        for value in var[1]:
            i[value] = i[var[0]].apply(lambda x: x==value)

In [6]:
truth = {'false':0.,'half-true':0.5,'mostly-true':0.75,'true':1.,'pants-fire':-0.25,'barely-true':0.25} 


for i in liar_list:
    i['numer_truth'] = i['label'].apply(lambda x: truth[x])

In [7]:
def score(file):
  data_pre = pd.read_csv(file, sep='\n', names = ["label"]) 
  score = f1_score(df_test['label'],data_pre, average='micro')
  return score

In [8]:

sid = SentimentIntensityAnalyzer()

ps = PorterStemmer()
wn = nltk.WordNetLemmatizer()

def clean_stem (sent, seq=False):
    temp1=sent
    temp2 = re.split('\W+',temp1)
    temp3 = [ps.stem(x) for x in temp2 if x not in stopwords]
    return temp3
def sentiment(x):
    score = sid.polarity_scores(x)
    return score['compound']

In [9]:
for i in liar_list:
    i['sentiment'] = i['statement'].apply(lambda x: sentiment(' '.join(clean_stem(x))))

In [10]:
df_test['sentiment'] = df_test['statement'].apply(lambda x: sentiment(' '.join(clean_stem(x))))

In [11]:
countV = CountVectorizer(analyzer='word', binary=True, decode_error='strict', encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
countV.fit_transform(df['statement'].values)



<11524x12590 sparse matrix of type '<class 'numpy.int64'>'
	with 114275 stored elements in Compressed Sparse Row format>

In [13]:

nb_cv = Pipeline([
        ('NBCV', countV),
        ('nb_clf', MultinomialNB(alpha=1.5, class_prior=None, fit_prior=True))])

In [ ]:
nb_cv.fit(df['statement'],df['label'])



Pipeline(steps=[('NBCV',
                 CountVectorizer(binary=True, max_df=0.5,
                                 stop_words='english')),
                ('nb_clf', MultinomialNB(alpha=1.5))])

In [ ]:
df.head(15)

,id,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,...,Governor,President-Elect,U.S. senator,Presidential candidate,Former governor,U.S. Representative,Senator,Milwaukee County Executive,numer_truth,sentiment
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,...,False,False,False,False,False,False,False,False,0.00,0.2960
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,...,False,False,False,False,False,False,False,False,0.50,0.0000
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,...,False,False,False,False,False,False,False,False,0.75,0.1280
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,...,False,False,False,False,False,False,False,False,0.00,0.8402
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,...,False,False,False,False,False,False,False,False,0.50,0.0000
5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,...,False,False,False,False,False,False,False,False,1.00,-0.2732
6,2342.json,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,...,False,False,False,False,False,False,False,False,0.25,0.0000
7,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,...,False,False,False,False,False,False,False,False,0.50,-0.0772
8,5602.json,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,...,False,False,False,False,False,False,False,False,0.50,0.0000
9,9741.json,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,...,False,False,False,False,False,False,False,False,0.75,0.0000


In [14]:
df2 = pd.concat([df_test, df_valid])

In [16]:
pd.concat([pd.DataFrame(nb_cv.predict(df2)), df2["statement"]])
df3 = pd.DataFrame()
df3["statement"] = df2["statement"]
df3["results"] = nb_cv.predict(df2["statement"])
df3["label"] = df2["label"]
df3["match"] = df3["label"] == df3["results"]
df3.groupby("match")["match"].count()




NotFittedError: This MultinomialNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [17]:
hold = pd.DataFrame()
hold["input"] = np.array(["Russian teen faces years in jail over social media post criticizing war in Ukraine"])
nb_cv.predict(hold["input"])[0]

NotFittedError: This MultinomialNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
import pickle
pickle.dump(nb_cv, open('model.pkl', 'wb'))

In [23]:
import pickle
loaded_model = pickle.load(open("model.pkl", 'rb'))
hold = pd.DataFrame()
hold["input"] = np.array(["Classified docs probe pushes Biden think tank into spotlight"])
loaded_model.predict(hold["input"])[0]


'half-true'